<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/getting-started/01-hello-world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AADC basic "Hello World" example
This example demonstrates how to use the AADC library to record a simple function and calculate its derivatives

In [1]:
import sys

### Please uncomment next line if you don't have AADC installed locally

In [2]:
#!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.27-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

In [3]:
import aadc
import numpy as np

# Simple function to record
The function can take arbitrary input types and can be defined in external file

In [4]:
def F(x,y,z):
    return np.exp(x / y + z)

# Object to hold recorded AADC Kernels
allow to speed up the calculation of the function itself and AAD for its derivatives


In [5]:
funcs = aadc.Functions()

# Define function arguments.
Note you need to use aadc.idouble instead of float

In [6]:
x = aadc.idouble(1.0)
y = aadc.idouble(2.0)
z = aadc.idouble(3.0)

# Trigger recording of the function

In [7]:
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


# Mark the function arguments as input and save reference argument ids

In [8]:
xArg = x.mark_as_input()
yArg = y.mark_as_input()
zArg = z.mark_as_input()

# Call the function and add some operations

In [9]:
f = F(x,y,z) + x

# Mark the result as output

In [10]:
fRes = f.mark_as_output()

# Stop recording
After this step AADC Kernel containing native machine CPU code will be generated and stored in the funcs object

In [11]:
funcs.stop_recording()

# Check if the function is recorded properly
So it can be used for arbitrary input values. This should return 0 if everything is OK, indicating that no branches in the function are not supported.


In [12]:
funcs.print_passive_extract_locations()

Number active to passive conversions: 0 while recording Python


# New samples of input values
To calculate the function and its derivatives at. Note that the x input is a vector of 20 samples and the y and z are scalars

In [13]:
inputs = {
    xArg:(1.0 * np.random.normal(1, 0.2, 20)),
    yArg:(2.0),
    zArg:(3.0),
}

Key: what output, value: what gradients are needed

In [14]:
request = {fRes:[xArg,yArg,zArg]}  

Run AADC Kernel for array inputs using 4 CPU threads and avx2

In [15]:
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

## Function output

In [16]:
Res[0][fRes]

array([32.08581563, 31.38530127, 31.62739357, 36.80152087, 33.69603014,
       29.05901561, 34.70163523, 39.10171386, 34.41085558, 27.75783793,
       34.35402649, 35.74232235, 33.15115711, 34.43709469, 38.82291664,
       33.6335681 , 36.29140435, 29.58695409, 30.00289371, 31.50297821])

## df/dx

In [17]:
Res[1][fRes][xArg]

array([16.60233975, 16.27339127, 16.38702499, 18.82690445, 17.36002691,
       15.18422985, 17.83425465, 19.91761619, 17.69704864, 14.57736753,
       17.67024091, 18.32581633, 17.10340089, 17.70942711, 19.7852397 ,
       17.33059644, 18.58548135, 15.43095436, 15.62553098, 16.32862024])

## df/dy

In [18]:
Res[1][fRes][yArg]


array([ -6.87389267,  -6.40351229,  -6.56520967, -10.23007577,
        -7.98349938,  -4.89750186,  -8.69595239, -11.97940531,
        -8.48843149,  -4.09427466,  -8.44801682,  -9.44854459,
        -7.60366624,  -8.50710751, -11.76366688,  -7.93973367,
        -9.85175277,  -5.23154837,  -5.49796933,  -6.48199625])